<a href="https://colab.research.google.com/github/kumar2612/ai_ml/blob/ai_ml_basics_nov2024/langgraph_tutorials/2_Build_an_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install langchain-community langgraph langchain-groq tavily-python langgraph-checkpoint-sqlite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9


In [5]:

from google.colab import userdata
import os

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]=userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"]="langchain_tutorials"

In [7]:
os.environ["TAVILY_API_KEY"]=userdata.get('TAVILY_API_KEY')

In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in BLR")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Bangalore', 'region': 'Karnataka', 'country': 'India', 'lat': 12.9833, 'lon': 77.5833, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1734005375, 'localtime': '2024-12-12 17:39'}, 'current': {'last_updated_epoch': 1734004800, 'last_updated': '2024-12-12 17:30', 'temp_c': 23.3, 'temp_f': 73.9, 'is_day': 1, 'condition': {'text': 'Light drizzle', 'icon': '//cdn.weatherapi.com/weather/64x64/day/266.png', 'code': 1153}, 'wind_mph': 7.8, 'wind_kph': 12.6, 'wind_degree': 35, 'wind_dir': 'NE', 'pressure_mb': 1014.0, 'pressure_in': 29.94, 'precip_mm': 1.64, 'precip_in': 0.06, 'humidity': 89, 'cloud': 75, 'feelslike_c': 25.7, 'feelslike_f': 78.3, 'windchill_c': 19.9, 'windchill_f': 67.8, 'heatindex_c': 19.9, 'heatindex_f': 67.8, 'dewpoint_c': 19.0, 'dewpoint_f': 66.3, 'vis_km': 3.0, 'vis_miles': 1.0, 'uv': 0.0, 'gust_mph': 10.7, 'gust_kph': 17.2}}"}, {'url': 'https://www.hindustantimes.com/cities/bengaluru-news/bangalore-wea

In [11]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.3-70b-versatile")

In [13]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content = "Hi")])

response.content

"It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [14]:
model_with_tools = model.bind_tools(tools)

In [15]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: I'm happy to chat with you, but I don't see a question or topic that you'd like to discuss. Is there something I can help you with or would you like to start a conversation?
ToolCalls: []


In [17]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in BLR?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'BLR weather'}, 'id': 'call_vepx', 'type': 'tool_call'}]


In [18]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [19]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='f11df159-5118-42d0-affa-7116edddcc5c'),
 AIMessage(content="I'm happy to chat with you, but I don't see a question that I can help with. Is there something I can assist you with? Perhaps you'd like to ask about current events?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 280, 'total_tokens': 321, 'completion_time': 0.149090909, 'prompt_time': 0.033918678, 'queue_time': 0.011323378000000002, 'total_time': 0.183009587}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_4e32347616', 'finish_reason': 'stop', 'logprobs': None}, id='run-9e100825-e6c9-4fe9-ba48-a917e8fe1981-0', usage_metadata={'input_tokens': 280, 'output_tokens': 41, 'total_tokens': 321})]

In [20]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='efa64907-97b6-414e-82a1-e3f603f76387'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bqyr', 'function': {'arguments': '{"query": "San Francisco weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 285, 'total_tokens': 306, 'completion_time': 0.076363636, 'prompt_time': 0.034753198, 'queue_time': 0.008756820999999998, 'total_time': 0.111116834}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c0cfa69934', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-81bae030-b75f-4686-b32e-1640f20a2ac1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather today'}, 'id': 'call_bqyr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 285, 'output_tokens': 21, 'total_tokens': 306}),
 ToolMessage(content='[

In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wsax', 'function': {'arguments': '{"query": "San Francisco weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 285, 'total_tokens': 305, 'completion_time': 0.072727273, 'prompt_time': 0.034640966, 'queue_time': 0.148609866, 'total_time': 0.107368239}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_4e32347616', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4f9ba261-24b1-4e98-b2fd-a6742508d319-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'call_wsax', 'type': 'tool_call'}], usage_metadata={'input_tokens': 285, 'output_tokens': 20, 'total_tokens': 305})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\'

In [22]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'San Francisco weather'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1734007399, \'localtime\': \'2024-12-12 04:43\'}, \'current\': {\'last_updated_epoch\': 1734006600, \'last_updated\': \'2024-12-12 04:30\', \'temp_c\': 12.2, \'temp_f\': 54.0, \'is_day\': 0, \'condition\': {\'text\': \'Light rain\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/296.png\', \'code\': 1183}, \'wind_mph\': 7.4, \'wind_kph\': 11.9, \'wind_degree\': 283, \'wind_dir\': \'WNW\', \'pressure_mb\': 1011.0, \'pressure_in\': 29.85, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 86, \'cloud\': 100, \'feelslike_c\': 11.0, \'feelslike_f\': 51.8, \'windchill_c\

In [23]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [24]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [25]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="It's nice to meet you, Bob. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 282, 'total_tokens': 307, 'completion_time': 0.090909091, 'prompt_time': 0.036159268, 'queue_time': 0.007977628, 'total_time': 0.127068359}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c0cfa69934', 'finish_reason': 'stop', 'logprobs': None}, id='run-3fc92bd8-69fa-4589-967e-c03897ece575-0', usage_metadata={'input_tokens': 282, 'output_tokens': 25, 'total_tokens': 307})]}}
----


In [26]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 321, 'total_tokens': 327, 'completion_time': 0.021818182, 'prompt_time': 0.043445668, 'queue_time': 0.008984985000000001, 'total_time': 0.06526385}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_4e32347616', 'finish_reason': 'stop', 'logprobs': None}, id='run-d2a399c3-461a-40f5-b79d-3cc62c34c8aa-0', usage_metadata={'input_tokens': 321, 'output_tokens': 6, 'total_tokens': 327})]}}
----


In [27]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't have any information about your name. If you'd like to share it with me, I can try to assist you further.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 283, 'total_tokens': 312, 'completion_time': 0.105454545, 'prompt_time': 0.032599873, 'queue_time': 0.007609646999999997, 'total_time': 0.138054418}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c0cfa69934', 'finish_reason': 'stop', 'logprobs': None}, id='run-1986cb7a-a2fc-4c8b-a82e-7ebcaad1176e-0', usage_metadata={'input_tokens': 283, 'output_tokens': 29, 'total_tokens': 312})]}}
----


In [28]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Can yo give me lnks to download NCERT class 1 book?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yf57', 'function': {'arguments': '{"query": "NCERT class 1 book download"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 336, 'total_tokens': 360, 'completion_time': 0.087272727, 'prompt_time': 0.040499897, 'queue_time': 0.0037505759999999985, 'total_time': 0.127772624}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c0cfa69934', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7fde1f13-4516-4fc0-a2f6-3a5b5c45d4c9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'NCERT class 1 book download'}, 'id': 'call_yf57', 'type': 'tool_call'}], usage_metadata={'input_tokens': 336, 'output_tokens': 24, 'total_tokens': 360})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.ncertbooks.guru/ncert-books-for-class-1/", "content": "NCERT Bo

In [29]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What are the subjects in it?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_x6h6', 'function': {'arguments': '{"query": "NCERT class 1 subjects"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 685, 'total_tokens': 708, 'completion_time': 0.083636364, 'prompt_time': 0.103760156, 'queue_time': 0.0069301529999999945, 'total_time': 0.18739652}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_4e32347616', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c5a0e5b4-8ff5-4691-9a88-02e5af88592e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'NCERT class 1 subjects'}, 'id': 'call_x6h6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 685, 'output_tokens': 23, 'total_tokens': 708})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.schoolmykids.com/education/cbse-class-1-syllabus", "content": "The Nationa